### map module

In [1]:
import sys
from numpy import mat , mean , power

In [2]:
def read_input(file):
    for line in file:
        yield line.rstrip()

In [3]:
input_ = read_input(sys.stdin)
input_ = [float(line) for line in input_]

numInputs = len(input_)

input_ = mat(input_)

sqInput = power(input_ , 2)

In [4]:
print '%d\t%f\t%f' % (numInputs , mean(input_) , mean(sqInput))

print >> sys.stderr , 'report:still alive'

0	nan	nan


I:\Anaconda3\envs\ipykernel_py2\lib\site-packages\numpy\matrixlib\defmatrix.py:536: RuntimeWarning: Mean of empty slice.
  return N.ndarray.mean(self, axis, dtype, out, keepdims=True)._collapse(axis)
I:\Anaconda3\envs\ipykernel_py2\lib\site-packages\numpy\core\_methods.py:73: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)
report:still alive


### reduce module

In [5]:
def read_input(file):
    for line in file:
        yield line.rstrip()

In [6]:
input_ = read_input(sys.stdin)

mapperOut = [line.split('\t') for line in input_]

cumVal = 0.0
cumSumSq = 0.0
cumN = 0.0

for instance in mapperOut:
    nj = float(instance[0])
    cumN += nj
    cumVal += nj*float(instance[1])
    cumSumSq += nj*float(instance[2])

mean = cumVal/cumN

varSum = (cumSumSq - 2*mean*cumVal + cumN*mean*mean)/cumN

print '%d\t%f\t%f' % (cumN , mean , varSum)

print >> sys.stderr , 'report: still alive'

ZeroDivisionError: float division by zero

### module mrjob

In [ ]:
from mrjob.job import MRJob

class MRmean(MRJob):
    def __init__(self , *args , **kwargs):
        super(MRmean , self).__init__(*args , **kwargs)
        self.inCount = 0
        self.inSum = 0
        self.inSqSum = 0.0
    
    def map_(self , key , val):
        if False:
            yield
        
        inVal = float(val)
        
        self.inCount += 1
        self.inSum += inVal
        self.inSqSum += inVal**2
        
    
    def map_final(self):
        mn = self.inSum/self.inCount
        mnSq = self.inSqSum/self.inCount
        
        yield (1 , [self.inCount , mn , mnSq])
        
    def reduce(self , key , packedValues):
        cumVal = 0.0
        cumSumSq = 0.0
        cumN = 0.0
        
        for valArr in packedValues:
            nj = float(valArr[0])
            cumN += nj
            cumVal += nj*float(valArr[1])
            cumSumSq += nj*float(valArr[2])
        
        mean = cumVal/cumN
        var = (cumSumSq-2*mean*cumVal + cumN*mean**2) / cumN
        
        yield (mean , var)
        
    def steps(self):
        return ([self.mr(mapper=self.map , reducer = self.reduce , mapper_final = self.map_final)])
    

#
MRmean.run()


In [7]:
#pegasos
def predict(w , x):
    return w*x.T

def batchpegasos(dataSet , labels , lam , T , k):
    m , n = shape(dataSet)
    w = zeros(n)
    
    dataIndex = range(m)
    
    for t in range(1 , T+1):
        wDelta = mat(zeros(n))
        eta = 1.0/(lam*t)
        random.shuffle(dataIndex)
        
        for j in range(k):
            i = dataIndex[j]
            p = predict(w , dataSet[i,:])
            
            if labels[i]*p < 1:
                wDelta += labels[i]*dataSet[i,:].A
        
        w = (1.0*1/t)*w+(eta/k)*wDelta
    
    return w